# Tutorial 7: Authoring APIs

In this tutorial, we'll do a more systematic walkthrough over the core authoring
APIs that you can use to define your workloads. Since this is a Python Jupyter
notebook, we'll default to Python, but the core abstractions are available also
in C++ and Java, albeit the exact syntax may vary. We encourate you to briefly
skim over the
[API documentation](https://github.com/google/genc/tree/master/genc/docs/api.md)
for an overview of the types of
APIs that exist in GenC, and how they vary across languages. In this tutorial,
our focus will be more hands-on and example-driven.

In the rest of the tutorial, we'll repeat some of the introductory material to
make it self-contained.

## Before we begin

As usual, make sure that you have a core development environment setup,
similarly to the preceding tutorials, so that you can load this notebook in
Jupyter and execute the code cells. For this, follow the basic instructions in
[SETUP.md](https://github.com/google/genc/tree/master/SETUP.md),
and reopen this notebook in Jupyter, and then execute the following code cells
to confirm that the setup works as expected.

In [ ]:
import genc
from genc.python import authoring
from genc.python import interop
from genc.python import runtime
from genc.python import examples

## Defining and running your first computation

Now, it's time to define your first computation. We're going to start with a
simple "hello, world" sort of example, a computation that uses a prompt
template to generate a greeting. You may write something as follows:

In [ ]:
@genc.python.authoring.traced_computation
def say_hello(name):
  return genc.python.authoring.prompt_template['Hello, {name}!'](name)

In order to run your computation, you're going to need to create an executor.
There are many ways to do it, but perhaps the simplest way to go is to register
a default executor we created, which enables you to call computations like the
one you've defined above similarly to how you'd call a Python function. You can
do this as follows:

In [ ]:
genc.python.examples.executor.set_default_executor()

Now you can run your computation:

In [ ]:
say_hello('Barbara')

You should see:

```
'Hello, Barbara!'
```

You can print the
[portable IR](https://github.com/google/genc/tree/master/genc/docs/ir.md)
that defines the computation you've just defined, as follows:

In [ ]:
print(say_hello.portable_ir)

With the IR at hand, there are more mechanisms at your disposal, such as running
your computation in a different programming language or execution environment.
These topics are outside of the scope of this authoring tutorial, but please see
[api.md](https://github.com/google/genc/tree/master/genc/docs/api.md)
for details if you're interested.

## The anatomy of your first computation

Now, let's take a closer look at the computation you've defined, and use it as
the opportunity to explain a few core concepts. As a reminder, here's what it
looked like:

```
@genc.python.authoring.traced_computation
def say_hello(name):
  return genc.python.authoring.prompt_template['Hello, {name}!'](name)
```

The first thing to note is that, you've defined it using a syntax similar to a
Python function, with a `traced_computation` decorator. The computation under
the hood is not actually Python code (as you saw above by printing the
`portable_ir` attribute, it's actually in the IR form), but it's constructed in
a Python-like fashion. In essence, code inside the decorated Python function
constructs the computation "graph" (so to speak), with each Python-like function
call inside adding a piece of that graph. The Python code inside it is being
traced at declaration time. In case you're familiar with the "graph mode" in
TensorFlow, the mechanism we use here is very similar.

Thus, in general, a definition of a computation using the tracing API that we're
showing here is going to look like the following. It can take one or more
arguments, which you can then refer to in the body of the computation. At some
point, you must use the `return` statement to indicate the result.

```
@genc.python.authoring.traced_computation
def say_hello(arg_1, arg_2, ..., arg_n):
  # Some code that might refer to any of the `arg_1`, `arg_2`, ..., `arg_n`
  return ...
```

In your example, you've also seen the use of a `prompt_template`, of the
building blocks we provide. Like most building blocks, it takes two parameters,
one in square brackets - this is called "static parameter", a baked-in
part of the computation logic, and one in round brackets - this is called the
"dynamic parameter", and it can be computed while your computation runs. In this
case, the prompt template text is static, but the name being fed to it is
dynamic, as it's supplied whren the computation is being invoked.

## Playing with prompt templates

Now that you understand the syntax, let's play with more this initial example,
still just using the prompt template as a building block, and illustrate a few
concepts in the process.

First, let's throw in another prompt template, so that your computation now
consists of two elements - the two prompt templates chained together, with one
feeding into the other:

In [ ]:
@genc.python.authoring.traced_computation
def say_hello(x):
  friendly_name = genc.python.authoring.prompt_template['my friend {name}'](x)
  return genc.python.authoring.prompt_template['Hello, {name}!'](friendly_name)

say_hello('Barbara')

You should see:

```
'Hello, my friend Barbara!'
```

First, notice that the names of the parameters to your computation and the
names of the parameters iun the prompt template are separate - there's no
connection between them:
*   The name `x` in the body of the computation represents the argument.
*   That argument `x` is being fed as an input into the first prompt template.
*   In the text of the first prompt template, the input is represented as a
    placeholders string `{name}`. Here, `{name}` simply represents the input.
    Any string will do.
*   We capture the output produced by the first promtp template, and give it a
    name `friendly_name` in the body of the traced computation.
*   This is subsequently fed as input to the second prompt template call.
*   The text of the second prompt template, again, happens to use `{name}` to
    represents its input. Any string would do.
*   The result of the second prompt template (and thus the entire chain) is
    being returned as the result of the computation.

The overall flow here is meant to resemble ordinary programming in Python, where
you intersperse function calls, and assign results to local variables (but keep
in mind the underlying logic is not Python, which has some implications, e.g.,
in terms of the order of execution - mor on this shortly).

With this, we're ready to expand on our example. Let's throw in an additional
argument to your function, and upgrade to use multi-variate prompt templates,
i.e., prompt templates that can take more than one parameter as input.

In [ ]:
@genc.python.authoring.traced_computation
def say_hello(title, first_name, last_name):
  full_name = genc.python.authoring.prompt_template_with_parameters[
      'my friend {x} {y}', ['x', 'y']](first_name, last_name)
  return genc.python.authoring.prompt_template_with_parameters[
      'Hello, {a}! Can I call you {b}? Or would you prefer {c} {d}?',
       ['a', 'b', 'c', 'd' ]](full_name, first_name, title, last_name)

say_hello('Sir', 'Alexander', 'Fleming')

You should see:

```
'Hello, my friend Alexander Fleming! Can I call you Alexander? Or would you
prefer Sir Fleming?'
```

In the above example, we used building block `prompt_template_with_parameters`,
which takes the additional list of parameter names. This list helps determine
the order in which parameters will be given, which may differ from the order in
which they're used in the text. Also, the same parameter can appear in the text
multiple times, like in this example:

In [ ]:
@genc.python.authoring.traced_computation
def show_off_counting(x, y, z):
  return genc.python.authoring.prompt_template_with_parameters[
      'I can count forward, check this out: {a}, {b}, {c}. And backwards, too: {c}, {b}, {a}.',
       ['a', 'b', 'c']](x, y, z)

show_off_counting('one', 'two', 'three')

You should see:

```
'I can count forward, check this out: one, two, three. And backwards, too: three, two, one.'
```

(...to be continued...)